# Building a Catalog

In [2]:
from classes.source_catalog import GetCnilCatalog

url = 'https://www.data.gouv.fr/api/1/organizations/534fff61a3a7292c64a77d59/catalog'
headers = {'accept': 'application/json'}
url_add = 'https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv'
instance1 = GetCnilCatalog(url, headers, url_add)
data = instance1.fetch_data_from_api()
data = data['@graph']
table_name = 'title'
download_url = 'downloadURL'
table_id = 'identifier' 
file_format= 'format'
last_update= 'modified' 
accessURL = '@id'
df_catalog = instance1.response_to_dataframe(data=data, table_name=table_name, download_url=download_url, table_id=table_id, file_format=file_format, last_update=last_update, accessURL=accessURL)
df_dataset = instance1.load_additional_info()
df_catalog = instance1.identify_datasets_info()
df_catalog = instance1.merge_additional_info()
instance1.save_to_csv('source_cnil_catalog')

Request is a success: 200
CSV file has been loaded to this path data/catalog/source_cnil_catalog_2024-02-10.csv


# Uploading CSV to GCS

In [2]:
from classes.file_to_gcs import FromFileToGCS
import os

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = ['data/catalog/source_cnil_catalog_2024-02-10.csv','data/catalog/opencnil-violationsdcpnotifiees-20230930.csv']
dest_folder = 'raw'
dest_blob = ['source_cnil_catalog_2024-02-10.csv', 'opencnil-violationsdcpnotifiees-20230930.csv']
init2.local_to_gcs(file_paths, dest_folder, dest_blob)

Bucket already exists.
Raw file source_cnil_catalog_2024-02-10.csv uploaded to GCS successfully to 2024-02-10/raw/source_cnil_catalog_2024-02-10.csv.
Raw file opencnil-violationsdcpnotifiees-20230930.csv uploaded to GCS successfully to 2024-02-10/raw/opencnil-violationsdcpnotifiees-20230930.csv.


In [4]:
bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
url = ['https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv', 'https://www.data.gouv.fr/fr/datasets/r/0f678674-4327-4c4d-8819-b6f508b41d0e']
dest_folder = 'raw'
dest_blob = ['datasets.csv', 'plaintes.csv']
init2.download_and_upload_from_URLs(url, dest_folder, dest_blob)

Bucket already exists.
Raw file datasets.csv downloaded and uploaded to GCS successfully to 2024-02-10/raw/datasets.csv.
Raw file plaintes.csv downloaded and uploaded to GCS successfully to 2024-02-10/raw/plaintes.csv.


# Downloading from Catalog

In [3]:
from classes.download_catalog_content import DlCatalogContent

instance3 = DlCatalogContent('data/catalog/source_cnil_catalog_2024-02-10.csv')
instance3.get_tables()

Error when downloading table Organismes ayant désigné un(e) délégué(e) à la protection des données (DPD/DPO) : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Notifications à la CNIL de violations de données à caractère personnel : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Sanctions prononcées par la CNIL : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Traitements de données personnelles déclarés à la CNIL depuis le 25 mai 2018 : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Protection des données personnelles dans le monde : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Contrôles réalisés par la CNIL : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Mises en demeure prononcées p

In [4]:
instance3.df_catalog

,index,table_id,table_name,download_URL,accessURL,data_format,last_update,dataset_id,dataset_name,frequency
0,0,a154f70f-ca8e-4d6a-8dcf-6d63662463eb,CNIL: les délibérations de la Commission natio...,https://echanges.dila.gouv.fr/OPENDATA/CNIL/,https://www.data.gouv.fr/datasets/53ca2dc6a3a7...,xml,2024-02-10T10:22:58.151456,53ca2dc6a3a7294a1ddd7848,les-deliberations-de-la-cnil,monthly
1,1,fbf9df01-36e9-4475-839d-4e63d869d2af,Les traitements de données personnelles déclar...,https://www.cnil.fr/fr/les-formalites-prealabl...,https://www.data.gouv.fr/datasets/555defefc751...,csv,2024-02-09T19:44:07,555defefc751df4b6ec98e0f,traitements-de-donnees-personnelles-declares-a...,weekly
2,2,b2f8954c-f0a4-4f60-a3eb-be55a0acf382,Formalités préalables reçues par la CNIL depui...,https://declarations.cnil.fr/opendata_declarat...,https://www.data.gouv.fr/datasets/5ef476d329a1...,xlsx,2024-02-05T02:57:35,5ef476d329a15f93d8a66bd1,traitements-de-donnees-personnelles-declares-a...,weekly
3,3,e7d67485-f5a8-4ad2-9554-4cb56e2643e2,Formalités préalables reçues par la CNIL depui...,https://declarations.cnil.fr/opendata_declarat...,https://www.data.gouv.fr/datasets/5ef476d329a1...,csv,2024-02-05T02:57:32,5ef476d329a15f93d8a66bd1,traitements-de-donnees-personnelles-declares-a...,weekly
4,4,5c926a7a634f410578005c68,Organismes ayant désigné un(e) délégué(e) à la...,NaN,https://www.data.gouv.fr/datasets/5c926a7a634f...,NaN,2024-01-30T10:06:46.769000,5c926a7a634f410578005c68,organismes-ayant-designe-un-e-delegue-e-a-la-p...,monthly
...,...,...,...,...,...,...,...,...,...,...
61,61,824481b4-449f-4a60-a093-390b2c788f07,Liste des contrôles réalisés par la CNIL en 2016,https://static.data.gouv.fr/resources/controle...,https://www.data.gouv.fr/datasets/555c431bc751...,xlsx,2017-03-30T17:04:19.852000,555c431bc751df5800190c78,controles-realises-par-la-cnil,annual
62,62,4a5883cd-7499-4b2f-a0fe-64c4f3088856,Liste des contrôles réalisés par la CNIL en 2015,https://static.data.gouv.fr/resources/controle...,https://www.data.gouv.fr/datasets/555c431bc751...,csv,2016-05-03T16:38:44.760000,555c431bc751df5800190c78,controles-realises-par-la-cnil,annual
63,63,b202d3a9-3ba8-44e7-8ca8-0a81d6421ce9,Liste des contrôles réalisés par la CNIL en 2015,https://static.data.gouv.fr/resources/controle...,https://www.data.gouv.fr/datasets/555c431bc751...,xlsx,2016-05-03T16:38:23.365000,555c431bc751df5800190c78,controles-realises-par-la-cnil,annual
64,64,440172f8-46b9-435c-a222-1b7c43c35c41,Liste des contrôles réalisés par la CNIL en 2014,https://static.data.gouv.fr/resources/controle...,https://www.data.gouv.fr/datasets/555c431bc751...,csv,2015-06-15T17:43:41.063000,555c431bc751df5800190c78,controles-realises-par-la-cnil,annual


# List all folders or all files

In [7]:
from classes.list_files import FolderLister

folder = 'data/datasets'
instance4 = FolderLister(folder)
fold_list= instance4.list_folders()
fold_list

['budget-de-la-cnil-1',
 'traitements-de-donnees-personnelles-declares-a-la-cnil-depuis-le-25-mai-2018',
 'marches-publics-de-la-cnil',
 'mises-en-demeure-prononcees-par-la-cnil',
 'effectifs-de-la-cnil',
 'les-deliberations-de-la-cnil',
 'droit-dacces-indirect-taj-stic-judex',
 'controles-realises-par-la-cnil',
 'plaintes-recues-par-la-cnil',
 'sanctions-prononcees-par-la-cnil',
 'protection-des-donnees-personnelles-dans-le-monde',
 'exercice-des-droits-indirect-donnees-generales',
 'correspondants-informatique-et-libertes-cil',
 'organismes-ayant-designe-un-e-delegue-e-a-la-protection-des-donnees-dpd-dpo',
 'traitements-de-donnees-personnelles-declares-a-la-cnil-avant-le-25-mai-2018',
 'notifications-a-la-cnil-de-violations-de-donnees-a-caractere-personnel']